# yorku_eecs_course_map
We wil scrape data off the YorkU website for selecting courses and create a graph so that students can see if they have prerequisites to take certain courses.
<br>This Python notebook will be used to collect the data.

### Import dependencies

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import time

In [90]:
def establish_connection_and_collect_data():
    global prerequisites
    COURSES_OFFERED = 92  # the eecs department at york offers 118 courses over f/w
    for i in range(COURSES_OFFERED):
        if i == 49 or i == 50 or i == 81:
            pass  # internship semsester for comp sci and security, cs projet
        else:
            # prepare url before making call to website
            url = 'https://w2prod.sis.yorku.ca/Apps/WebObjects/cdm.woa/5/wo/1AxnNjuye9XHuJ3uAR15R0/2.3.10.8.3.'
            url += str(i) + '.0.5'

            ''' make call to website '''
            # establish connection and download html file
            u_client = urlopen(url)
            # offload contents into variable
            page_html = u_client.read()
            # close connection with client
            u_client.close()

            ''' parse html via BS '''
            page_soup = soup(page_html, 'html.parser')
            course_title = str(page_soup.find('font', {'color': '#CC0000'}))
            course_title  = course_title[22:-7]
            course_description = page_soup.find('table', {'cellpadding': '10'}).tr.td.find_all('p')
            course_description = course_description[3].text

            # course information
            course_code = course_title[:12]
            course_credit = course_title[15:19]
            course_name = course_title[22:]
            prerequisite_description = ''
            course_corequisites = None

            if 'prerequisites' in course_description.lower():
                # two ways information is presented
                # Prerequisites: General Prerequisite; LE/EECS 1030 3.00 or LE/EECS 2030 3.00. 
                # Prerequisites: SC/MATH 1190 3.00, or two 4U Math courses
                start_index = course_description.lower().index('prerequisites') + 14  # including the ':' after the word
                if 'general prerequisite' in course_description.lower():
                    start_index = course_description.lower().index('general prerequisite') + 21
                end_index = len(course_description) - 1  # to account for '.' @ end of the sentence
                if 'course credit exclusion' in course_description.lower():
                    end_index = course_description.lower().index('course credit exclusion')
                if 'corequisites' in course_description.lower():
                    end_index = course_description.lower().index('corequisites') - 2 # account for '.' @ end of the sentence
                    course_corequisites = course_description[course_description.lower().index('corequisites') + 13:]
                prerequisite_description = course_description[start_index:end_index]
                # THE CODE ABOVE HAS BEEN CHECKED AND WORKS AS IT WAS INTENDED

            elif 'prerequisite' in course_description.lower():
                # some have: Prerequisite: SC/MATH 1013 3.00 or equivalent;
                start_index = course_description.lower().index('prerequisite') + 13  # including the ':' after the word
                if 'general prerequisite' in course_description.lower():
                    start_index = course_description.lower().index('general prerequisite') + 21
                end_index = len(course_description) - 1  # to account for '.' @ end of the sentence
                if 'course credit exclusion' in course_description.lower():
                    end_index = course_description.lower().index('course credit exclusion') - 2
                if 'corequisite' in course_description.lower():
                    end_index = course_description.lower().index('corequisite') - 2 # account for '.' @ end of the sentence
                    some_index = course_description.lower().index('course credit exclusion')-2
                    course_corequisites =  course_description[course_description.lower().index('corequisite') + 13:some_index]
                prerequisite_description = course_description[start_index:end_index]
                # THE CODE ABOVE HAS BEEN CHECKED AND WORKS AS IT WAS INTENDED

            else:
                prerequisite_description = 'None'

            # THE DATA COLLECTION STEP IS NOW COMPLETE! DO SOME DATA WRANGLING BEFORE MAKING VISUALIZATION

            wrangle(course_code, course_name, course_credit, prerequisite_description, course_corequisites, url)

        
        
establish_connection_and_collect_data()

None
https://w2prod.sis.yorku.ca/Apps/WebObjects/cdm.woa/5/wo/1AxnNjuye9XHuJ3uAR15R0/2.3.10.8.3.0.0.5 

None
https://w2prod.sis.yorku.ca/Apps/WebObjects/cdm.woa/5/wo/1AxnNjuye9XHuJ3uAR15R0/2.3.10.8.3.1.0.5 

One of (1)-(3) below must be met: (1) (New high school curriculum): One 4U Math course with a grade of at least 75%. (2) Completion of 6.00 credits from York University MATH courses (not courses with second digit 5) with a grade point average of 5.00 (C+) or better over these credits; (3) Completion of 6.00 credits from York University mathematics courses whose second digit is 5, with an average grade not below 7.00 (B+)
https://w2prod.sis.yorku.ca/Apps/WebObjects/cdm.woa/5/wo/1AxnNjuye9XHuJ3uAR15R0/2.3.10.8.3.2.0.5 

SC/MATH 1190 3.00, or two 4U Math courses, MHF4U (Advanced Function). Course credit  exclusions: LE/EECS 1028 3.00, SC/MATH 1028 3.00, SC/MATH 2320 3.00
https://w2prod.sis.yorku.ca/Apps/WebObjects/cdm.woa/5/wo/1AxnNjuye9XHuJ3uAR15R0/2.3.10.8.3.3.0.5 

LE/EECS1011 3.00

In [88]:
#57, #81: passed 40/24 creds, 87 co-req,
def wrangle(course_code, course_name, course_credit, prerequisite_description, course_corequisites, url):
    prerequisite_description = prerequisite_description.strip()
    
    # remove the gpa notice
    maintain_gpa = False
    if '(NOTE:' in prerequisite_description:
        index = prerequisite_description.index('(NOTE')
        prerequisite_description = prerequisite_description[:index]
        maintain_gpa = True
    
    # remove course instructor notice - 2 ways
    # Permission of the course director is needed
    # permission of the course director. Some more text
    permission_required = False
    if 'permission' in prerequisite_description.lower():
        index = prerequisite_description.lower().index('permission')
        prerequisite_description = prerequisite_description[:index]
        permission_required = True
    
    # remove: students with background seek help
    if 'Note: Students with background' in prerequisite_description:
        index = prerequisite_description.index('Note: Students with background')
        prerequisite_description = prerequisite_description[:index]
        permission_required = True
    
    # remove: (with minimum grade of B)
    if 'successful completion' in prerequisite_description:
        index = prerequisite_description.index('successful completion')
        prerequisite_description = prerequisite_description[:index]
        permission_required = True
    
    # remove: inclusing LE/EECS XXXX, LE/EECS XXXX
    if ' including ' in prerequisite_description:
        index = prerequisite_description.index(' including ')
        prerequisite_description = prerequisite_description[:index] + prerequisite_description[index+10:]
    
    # remove: (with minimum grade of B)
    minimum_b_required = False
    if '(with minimum grade of B)' in prerequisite_description:
        index = prerequisite_description.index('(with minimum grade of B)')
        prerequisite_description = prerequisite_description[:index] + prerequisite_description[index+25:]
    
    prerequisite_description = prerequisite_description.strip()
    if prerequisite_description[-1] is '.' or prerequisite_description[-1] is ';':
        prerequisite_description =  prerequisite_description[:-1]
    
    print(prerequisite_description)
    print(url, '\n')

In [85]:
"""
TODO:
    EXTRACT COURSE CREDIT EXCLUSIONS
    THEN GRAPH
    ------------------------------
    REGULAR EXPRESSIONS!!!

"""

25